# **라이브러리 정의**

# **라이브러리 정의**

In [1]:
import os
import pandas as pd
from supabase import create_client, Client

# **사용자 정보 초기화**

In [2]:
# 엔드포인트 URL 및 API 키 설정
SUPABASE_URL = 'https://yhayrbotkkuuvoxzhqct.supabase.co'
SUPABASE_KEY = 'sb_secret_5rUltxbkuiB3wFcTyMs1qw_cJHFo3kf'

# 클라이언트 연결
try:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Supabase 클라이언트 연결 완료.")
except Exception as e:
    print(f"Supabase 클라이언트 연결 실패: {e}")

def validation(response):
    if response.data:
        print("데이터베이스 적용 완료")
    else:
        print("코드를 재확인해주세요.")

Supabase 클라이언트 연결 완료.


# **기본 문법**
: 모든 쿼리 로직 정의 이후 최종적으로 결과물을 얻으려면 아래 메서드를 이어붙여야함
- .execute().data

## **테이블 및 컬럼 선택**

In [3]:
# Query : select * from quiz

# 1. 테이블 선택 (from)
select_table = supabase.from_('quiz')

# 2. 컬럼 선택 (select)
select_table_column = select_table.select('*')

## **특정 값 필터 : 단수**

In [4]:
# 3-1. 특정 값 필터 (단수)
third = select_table_column.eq('category', 'Basic')

df = pd.DataFrame(third.execute().data)
df.head(2)

,category,sub_category,question,answer,explanation
0,Basic,배당금,배당수익률은 배당금을 주가로 나눈 비율이다.,O,배당수익률 = 주당 배당금 / 주가 × 100
1,Basic,주식,주식은 은행에서만 거래할 수 있다.,X,"주식은 증권사를 통해 거래하며, 은행은 주식 거래 서비스를 제공하지 않는다."


## **특정 값 필터 : 복수 (AND)**

In [5]:
# 3-2. 특정 값 필터 (복수 : AND)
third_and = select_table_column.eq('category', 'Basic').eq('sub_category', '배당금')

df = pd.DataFrame(third_and.execute().data)
df.head(2)

,category,sub_category,question,answer,explanation
0,Basic,배당금,배당수익률은 배당금을 주가로 나눈 비율이다.,O,배당수익률 = 주당 배당금 / 주가 × 100
1,Basic,배당금,보통주는 배당금이 정해져 있다.,X,보통주는 배당금이 정해져 있지 않고 기업 정책에 따라 달라진다.


## **특정 값 필터 : 복수 (OR)**
- 'category.eq.Basic' : category 컬럼의 값이 'Basic'
- 'answer.eq.O' : answer 컬럼의 값이 'O'

- 비교 연산자 : eq(=), neq(!=), gt(>), gte(>=), lt(<), lte(<=)

In [6]:
# 3-4. 특정 값 필터 (복수 : OR)

# 조건 정의
condition = 'category.eq.Basic, answer.eq.O'

# .or_()
third_or = select_table_column.or_(condition)

df = pd.DataFrame(third_or.execute().data)

df.head(2)

,category,sub_category,question,answer,explanation
0,Basic,배당금,배당수익률은 배당금을 주가로 나눈 비율이다.,O,배당수익률 = 주당 배당금 / 주가 × 100
1,Basic,배당금,보통주는 배당금이 정해져 있다.,X,보통주는 배당금이 정해져 있지 않고 기업 정책에 따라 달라진다.


## **정렬 : 오름차순 / 내림차순**
: 랭킹 or 게시판 시스템에 사용하기 유용

In [7]:
response = supabase.from_('quiz').select('*').order('sub_category', desc = True).limit(2).execute().data

df = pd.DataFrame(response)
df.head(2)

,category,sub_category,question,answer,explanation
0,financial,현금흐름표,현금흐름표는 기업의 자산 증가 여부를 나타낸다.,X,현금흐름표는 실제 현금 유입과 유출을 보여주는 보고서다.
1,financial,현금흐름표,현금흐름표는 투자자산 증가 여부를 직접적으로 나타낸다.,O,투자활동현금흐름 항목을 통해 확인 가능하다.


# **행 추가**

In [8]:
# 행 정보 입력 -> 컬럼명: 내용
insert_row = {
    'data_code': 'financial_현금흐름',
    'type': 'financial',
    'title': '현금흐름',
    'markdown_text': '현금흐름은 테스트로 넣은 임시 정보입니다.'
}

response = supabase.table('education_data').insert(insert_row).execute()

validation(response)

데이터베이스 적용 완료


# **행 삭제**

In [9]:
# 특정 행 삭제
response = supabase.table('education_data').delete().eq('data_code', 'financial_현금흐름').execute()

validation(response)

# 여러 행 삭제
# delete_primary_keys = ['financial_현금흐름', 'financial_임시데이터']

# supabase.table('education_data').delete().eq('data_code', delete_primary_keys).execute()

데이터베이스 적용 완료


# **행 안의 데이터 수정**
- table(): 수정할 테이블
- update(): 업데이트할 내용 -> {컬럼명: 내용}
- eq() : update가 적용될 대상 -> Key, Value

In [10]:
# 데이터 수정 (테스트)
update_value = {
    'title': 'Test update'  # title 컬럼을 test update로 변환
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

데이터베이스 적용 완료


In [11]:
# 데이터 원복
update_value = {
    'title': '재무상태표'
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

데이터베이스 적용 완료


In [5]:
user_info = supabase.from_('user_info').select('id', 'pw').execute().data

df = pd.DataFrame(user_info)

df.head()

,id,pw
0,gildong_hong,$2b$12$IRFempd4zfVRSWUF80bgH.rx0VHehhTG69rCJM8...
